# Besoin2

## implémentation random forest from scratch

## CART

In [15]:
import numpy as np
from collections import Counter

#calcul des critères de division

# Définition de la fonction gini pour la classifiction pour un ensemble de labels y
def gini(y):
    m = len(y)
    return 1.0 - sum((np.sum(y == c) / m) ** 2 for c in np.unique(y))

#ou variance pour la régression
def variance(y):
    return np.var(y)

# Définition de la fonction split de divisions des noeuds
#fonction pour diviser les données basées sur une caractéristique et un seuil
def split(X, y, feature_index, threshold):
    left_indices = np.where(X[:, feature_index] <= threshold)[0]
    right_indices = np.where(X[:, feature_index] > threshold)[0]
    return X[left_indices], X[right_indices], y[left_indices], y[right_indices]

#Construction de l'arbre de décision 

# Définition de la classe Node: noeud de l'abre de décision
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index  #index de la caractéristique utilisée pour la division
        self.threshold = threshold          #seuil de division
        self.left = left                    #sous-arbre gauche
        self.right = right                  #sous-arbre droit
        self.value = value                  #valeur de la feuille (si c'est une feuille)


# Définition de la classe DecisionTree responsable de la construction et de l'utilisation de l'arbre de décision
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=2, criterion='gini'): 
        self.root = None #racine de l'abre
        self.min_samples_split = min_samples_split #nombre minimum d'échantillons pour diviser un nœud
        self.max_depth = max_depth #profondeur maximale de l'arbre
        self.criterion = criterion #critère de division (ici nous utilisons Gini)
    
    def fit(self, X, y):
        self.n_classes = len(set(y)) #nombre de classes uniques 
        self.root = self._grow_tree(X, y)#construction de l'arbre
    
    def _grow_tree(self, X, y, depth=0):
        # vérifie plusieurs conditions pour déterminer s'il faut arrêter la croissance de l'arbre et créer une feuille
        n_samples, n_features = X.shape #initialisation
        #critère d'arret
        if (depth >= self.max_depth or n_samples < self.min_samples_split or len(set(y)) == 1):
            #critères de profondeur maximale, de nombre minimumd'echantillons, appartenance des classes
            leaf_value = self._most_common_label(y) #si l'un des critères est satisfait, une feuille est créé
            return Node(value=leaf_value)
        
        # trouver la meilleure division (best split)
        best_feature, best_threshold = self._best_split(X, y, n_features)
        
        #division des données
        left_X, right_X, left_y, right_y = split(X, y, best_feature, best_threshold)
       
        #construire les sous arbres par récursivité 
        left_child = self._grow_tree(left_X, left_y, depth + 1)
        right_child = self._grow_tree(right_X, right_y, depth + 1)
        return Node(best_feature, best_threshold, left_child, right_child)
    
    #trouve la meilleure caractéristique et le meilleur seuil pour diviser les données
    def _best_split(self, X, y, n_features):
        #initialisation des variables
        best_gini = 1.0 #initialisation à 1 soit la pire valeur possible pour l'impureté de Gini
        split_idx, split_thresh = None, None # stockeront l'index de la caractéristique et  le seuil correspondant à la meilleure division
        
        
        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index]) #tableau des valeurs uniques dans la colonne feature_index de X
            for threshold in thresholds: #boucle parcourt chaque valeur unique (seuil) pour la caractéristique courante
                gini = self._gini_index(X, y, feature_index, threshold)
                #mise à jour de la meilleure division pour les 3 variables
                if gini < best_gini: #Si l'impureté de Gini pour la division courante est meilleure que best_gini, on met à jour best_gini, split_idx et split_thresh
                    best_gini = gini
                    split_idx = feature_index
                    split_thresh = threshold
        return split_idx, split_thresh
    
    def _gini_index(self, X, y, feature_index, threshold): #calcule l'impureté de Gini pour une division donnée par feature_index et threshold
       #on divise les indices 
        left_indices = np.where(X[:, feature_index] <= threshold)[0] #indices des échantillons pour lesquels la valeur de la caractéristique est inférieure ou égale au seuil
        right_indices = np.where(X[:, feature_index] > threshold)[0] # indices des échantillons pour lesquels la valeur de la caractéristique est supérieure au seuil
        #vérification des divisions vides
        if len(left_indices) == 0 or len(right_indices) == 0:#si une des divisions (gauche ou droite) est vide-->
            return 1.0  #retourner 1.0 =la pire impureté de Gini (division inutile)
       
        #on calcule de l'Impureté de Gini pour chaque division
        left_gini = gini(y[left_indices]) # impureté de Gini pour les labels des échantillons dans la division gauche
        right_gini = gini(y[right_indices])# "" droite
        #calcul des poids pour les divisions
        n_left, n_right = len(left_indices), len(right_indices)
        #calcul de l'impureté de Gini pondérée par le nombre d'echantillons dans chaque division
        weighted_gini = (n_left * left_gini + n_right * right_gini) / (n_left + n_right)
        
        return weighted_gini
    
    def _most_common_label(self, y):
        return Counter(y).most_common(1)[0][0] #retourne le label le plus fréquent dans un ensemble de labels y
    
    def predict(self, X):#utilise l'arbre de décision pour faire des prédictions sur un ensmeble de nouvelles données X
        return np.array([self._predict(inputs) for inputs in X]) #applique la fonction _predict sur chaque exemple d'entrée dans X et retourne un tableau des prédictions
    
    def _predict(self, inputs):# traverse l'arbre de décision pour prédire la classe pour un exemple d'entrée donné
        node = self.root #initialisation
        #on traverse l'arbre
        while node.value is None: #Tant que le nœud courant n'est pas une feuille (n'a pas de valeur de classe), la fonction vérifie la valeur de la caractéristique de inputs correspondant à node.feature_index
            if inputs[node.feature_index] <= node.threshold: #si cette valeur est inférieure ou égale à node.threshold elle se déplace vers le sous-arbre gauche (node.left), sinon elle se déplace vers le sous-arbre droit (node.right).
                node = node.left
            else:
                node = node.right
        return node.value #retourne la prédiction

# test avec de données random pour l'exemple
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = make_classification(n_samples=1000, n_features=20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#on évalue des performances
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")



Accuracy: 0.875


## random forest

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

class RandomForestClassifier:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2, max_features=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.trees = []
        self.selected_features = []  # Ajouter un attribut pour stocker les caractéristiques sélectionnées

    def fit(self, X, y):
        n_samples, n_features = X.shape
        if not self.max_features:
            self.max_features = int(np.sqrt(n_features))
        
        for _ in range(self.n_estimators):
            # Bootstrap sampling
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_bootstrap = X[indices]
            y_bootstrap = y[indices]

            # Randomly select features
            selected_features = np.random.choice(n_features, self.max_features, replace=False)
            self.selected_features.append(selected_features)  # Sauvegarder les caractéristiques sélectionnées

            # Create decision tree using DecisionTreeClassifier from scikit-learn
            tree = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X_bootstrap[:, selected_features], y_bootstrap)

            # Add tree to forest
            self.trees.append(tree)

    def predict(self, X):
        # Aggregate predictions from each tree
        predictions = np.zeros((X.shape[0], len(self.trees)))
        for i, tree in enumerate(self.trees):
            selected_features = self.selected_features[i]  # Récupérer les caractéristiques sélectionnées pour cet arbre
            predictions[:, i] = tree.predict(X[:, selected_features])  
        
        # For classification, use majority vote
        return np.round(np.mean(predictions, axis=1)).astype(int)

# Exemple d'utilisation

# Génération de données synthétiques pour l'exemple
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Création du modèle de Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, max_features=None)

# Entraînement du modèle sur les données d'entraînement
rf_classifier.fit(X_train, y_train)

# Prédictions sur les données de test
y_pred = rf_classifier.predict(X_test)

# Évaluation des performances
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.875
